Wahlpflichtfach Künstliche Intelligenz I: Praktikum

---

# 03 - Kontrollfluss und Objektorientierung

- [Funktionen](#Funktionen)
- [Klassen](#Klassen)
- [Typisierung](#Typisierung)
- [The Python Data Model](#The-Python-Data-Model)
- [Iterables und Iteratoren](#Iterables-und-Iteratoren)
- [Properties](#Properties)
- [Exceptions](#Exceptions)
- [Factory-Methoden](#Factory-Methoden)
- [Generators](#Generators)
- [Kontextmanager (und IO)](#Kontextmanager-(und-IO))
- [Map, Filter & Reduce](#Map,-Filter-&-Reduce)
- [Decorators](#Decorators)
- [Module](#Module)

#### Erzeugen von Objekten

Objekte einer Klasse meineKlasse können durch den Aufruf:

```python
ref=meineKlasse(arg1,arg2,...)
```

erzeugt werden. Die innerhalb der geschweiften Klammern angeführten Argumente werden dem Konstruktor übergeben. Enthält der Konstruktor als Parameter nur self, dann können die geschweiften Klammern nach dem Klassennamen ganz weggelassen werden.

In [ ]:
class Konto:
        angelegteKonten=0

        def __init__(self, inhaber, autorisiert=["Bankangestellter"], startkap=0):
            self.__inhaber = inhaber
            self.__autorisiert = autorisiert
            self.__kontostand = startkap
            Konto.angelegteKonten += 1

        def __del__(self):
            Konto.angelegteKonten -= 1

        def einzahlen(self,betrag):
            if isinstance(betrag, (float, int)) and betrag>0:
                self.__kontostand +=betrag
                print("Neuer Kontostand:    ", self.__kontostand)
            else:
                print("FEHLER: Falsche Betragsangabe")
            return self.__kontostand

        def auszahlen(self, betrag,initiator):
            if not initiator in self.__autorisiert:
                print(f"{initiator} ist nicht berechtigt")
            elif self.__kontostand < betrag:
                print(f"Es befinden sich nur noch {self.__kontostand:10.2f} Euro auf dem Konto")
            else:
                self.__kontostand -= betrag
            return self.__kontostand

        def abfrage(self):
            return self.__kontostand

In [ ]:
kontoSchwarz=Konto("Schwarz",["Schwarz","Bankangestellter","Papa"],10)
print(kontoSchwarz.einzahlen(1499.00))
print(kontoSchwarz.auszahlen(1000, "Freundin"))
print(kontoSchwarz.abfrage())
print(kontoSchwarz.auszahlen(1600, "Papa"))
print(kontoSchwarz.auszahlen(900, "Schwarz"))

print("Anzahl der Konten  :",Konto.angelegteKonten)

kontoWeiss=Konto("Weiss")

print("Anzahl der Konten  :",Konto.angelegteKonten)

print("Kontostand von Schwarz :",kontoSchwarz.abfrage())
print("Kontostand von Weiss :",kontoWeiss.abfrage())

del(kontoWeiss)

print("Anzahl der Konten  :",Konto.angelegteKonten)

In [ ]:
class GiroKonto(Konto):
    pass

In [ ]:
kontoPhilipp = GiroKonto("Philipp", ["Philipp", "Bankangestellter"], 10)
print(kontoPhilipp.einzahlen(1000.00))

## Fortgeschrittenes Python

Bis jetzt haben Sie das **Bread and Butter** der Programmierung in Python kennengelernt. Sie können jetzt mit einfachen Datentypen, grundlegenden Operatoren, Datensammlungen und Kontrollfluss umgehen. Außerdem wissen Sie, wie Sie in Python Schleifen erstellen und Codeabschnitte mit Funktionen wiederverwenden können. In diesem Abschnitt sehen wir uns eine Reihe von Spracheigenschaften an, die Python im Vergleich zu anderen Programmiersprachen besonders machen. 

Alle Datentypen sind Objekte?! Zumindest sind alle eine Instanz von `object`:

In [ ]:
isinstance(2, object), isinstance(2.0, object), isinstance(True, object), 

Objekte haben normalerweise Methoden. Selbst etwas so Einfaches wie ein Boolean hat eine Menge Methoden:

In [ ]:
dir(True)

## Typisierung

Python ist dynamisch typisiert, aber stark typisiert. 

Stellen Sie sich diesen Code vor:
```
variable = 3
variable = 'hallo'
```
Hier hat sich `variable` nicht geändert - denn `variable` ist kein Objekt, sondern einfach ein Name. In der ersten Zeile ist der Name `variable` an ein `int`-Objekt gebunden, und in der Zeile danach ist derselbe Name an ein `string`-Objekt gebunden -- Python ist dynamisch typisiert, weil wir Referenzen herumreichen und den Typ erst in der letzten möglichen Minute überprüfen. Wir sagen, es ist stark typisiert, weil Objekte ihren Typ nicht ändern.

Um den Typ eines Objekts zu ermitteln, verwenden Sie die Funktion `type`. Wir können Variablen so oft neu zuweisen, wie wir wollen, mit jedem Typ, den wir wollen! Das ist ein Beispiel für dynmaische Typisierung:

In [ ]:
variable = 3
print(type(variable))
variable = 'hallo'
print(type(variable))

## The Python Data Model

Wir haben bereits Mathe-Operatoren auf Zahlen gesehen:

In [ ]:
# Standard math operators work as expected on numbers
a = 2
b = 3

print('a + b = ', a + b)
print('a - b = ', a - b)
print('a * b = ', a * b)
print('a ** b = ', a ** b)  # a to the power of b (a^b is a bit-wise XOR!)
print('a / b = ', a / b)
print('a // b = ', a // b)  # Floor division 
print('b % a = ', b % a)    # Modulo operator (divide, return remainder)

Und wir haben diese Operatoren auch bei Strings gesehen!

In [ ]:
print('hello' + 'world')
print('hello' * 3)

Woher weiß Python also, *welches davon* es verwenden soll?  
  
Unter der Haube ist die meiste Python-Syntax nur *syntatic sugar* für Methodenaufrufe. 

Wenn Sie also aufrufen...

In [ ]:
mylist = [1, 2, 3]
len(mylist)

Was Python daraus macht, ist eigentlich:

In [ ]:
mylist.__len__()

Also, unter der Haube von Python ist im Grunde folgendes:

In [ ]:
def len(obj):
    return obj.__len__()

Wenn Sie die Methoden kennen, die von der allgemeinen Syntax implizit aufgerufen werden, können Sie Objekte entwerfen, die sich wunderbar in die Sprache integrieren.

Ein weiteres Beispiel:

In [ ]:
3 + 3 

... macht in Wirklichkeit:

In [ ]:
(3).__add__(3)

Lassen Sie uns herausfinden, wie wir Objekte erstellen können, die sich gut mit dem Rest der Sprache verhalten. Als Beispiel schauen wir uns eine Klasse zur Darstellung von Tripeln numerischer Werte an.

In [ ]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3

In [ ]:
Triple(1, 2, 3)

In [ ]:
range(1, 5)

Die String-Darstellung unserer Klasse ist nicht wirklich informativ. Wir können dies beheben, indem wir `__repr__` implementieren.

In [ ]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return f"Triple({self.nums[0]}, {self.nums[1]}, {self.nums[2]})"
    
Triple(1, 2, 3)

Um die Addition zwischen `Triple`s zu ermöglichen, müssen wir `__add__` implementieren. Wir definieren die Addition `Triple`s genauso wie die elementweise Addition der drei Zahlen.  

In [ ]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return f"Triple({self.nums[0]}, {self.nums[1]}, {self.nums[2]})"
    
    def __add__(self, other):
        num1 = self.nums[0] + other.nums[0]
        num2 = self.nums[1] + other.nums[1]
        num3 = self.nums[2] + other.nums[2]
        return Triple(num1, num2, num3)
        
    
a = Triple(1, 2, 3)
b = Triple(2, 3, 4)

Da wir `__add__` implementiert haben, können wir Tripel mit dem Plus-Operator addieren. Die folgenden drei Ausdrücke sind alle gleich! Der erste ist der schnelle Weg, der intern auf den zweiten abbildet, der wiederum intern auf den dritten abbildet!

In [ ]:
print(a + b)
print(a.__add__(b))
print(Triple.__add__(a, b))

In [ ]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return f"Triple({self.nums[0]}, {self.nums[1]}, {self.nums[2]})"
    
    def __add__(self, other):
        if isinstance(other, Triple):
            num1 = self.nums[0] + other.nums[0]
            num2 = self.nums[1] + other.nums[1]
            num3 = self.nums[2] + other.nums[2]
            return Triple(num1, num2, num3)
        elif isinstance(other, int):
            return Triple(self.nums[0] + other, self.nums[1] + other, self.nums[2] + other)
        else:
            return NotImplementedError
        
    
a = Triple(1, 2, 3)
b = Triple(2, 3, 4)

In [ ]:
a + 1

Nun wäre es schön, die Addition zwischen Tripel und skalaren Zahlen zu ermöglichen, indem man den Skalar einfach elementweise zu allen drei Tripelwerten addiert. Aber der Ausdruck

In [ ]:
1 + Triple(1, 2, 3)

... wird interpretiert als ...

In [ ]:
int.__add__(1, Triple(1, 2, 3))

... der den speziellen Wert `NotImplemented` zurückgibt. Wenn eine Binäroperation beim Aufruf auf dem ersten Operanden nicht funktioniert, versucht Python, die Reihenfolge der Operanden umzukehren, indem es `__radd__` auf dem anderen aufruft. Wenn auch dies nicht funktioniert, wird ein `TypeError` ausgelöst. Durch die Implementierung von `__radd__` können wir die skalare Addition zum Laufen bringen, ohne den Behaivor der `int`s zu ändern.

In [ ]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return f"Triple({self.nums[0]}, {self.nums[1]}, {self.nums[2]})"
    
    def __add__(self, other):
        if isinstance(other, Triple):
            num1 = self.nums[0] + other.nums[0]
            num2 = self.nums[1] + other.nums[1]
            num3 = self.nums[2] + other.nums[2]
            return Triple(num1, num2, num3)
        elif isinstance(other, int):
            return Triple(self.nums[0] + other, self.nums[1] + other, self.nums[2] + other)
        else:
            return NotImplementedError
    
    def __radd__(self, other):
        num1 = self.nums[0] + other
        num2 = self.nums[1] + other
        num3 = self.nums[2] + other
        return Triple(num1, num2, num3)
    
    def __contains__(self, value):
        return value in self.nums

In [ ]:
1 + Triple(1, 2, 3)

In [ ]:
Triple(1, 5, 7) + 2

In [ ]:
4 in Triple(1, 2, 3)

2 in Triple(1, 2, 3) 

Sie können mehr über das Datenmodell von Python unter https://docs.python.org/3/reference/datamodel.html lesen.

**Wiederholen Sie für sich selber das gehörte"**

Bearbeiten Sie inbesondere die folgende **Übung** und schreiben Sie die Antwort am Ende der Bearbeitungszeit in den Chat: 

Machen Sie den "in"-Operator auf unsere Tripel anwendbar. Dazu müssen wir `__contains__` implementieren. Die Anweisung 

```python 
3 in Tripel(1, 2, 3)
```

sollte nach der Definition von `__contains__` zu `True` werden.

### Wahrheitswertprüfung

Jedes Objekt kann auf Wahrheitswert getestet werden, zur Verwendung in einer `if`- oder `while`-Bedingung oder als Operand der booleschen Operationen. Die folgenden Objekte werden als falsch angesehen:
* `None`
* `False`
* Nullen von numerischen Typen (`0`, `0.0`)
* Leere Sequenzen und Sammlungen: `''`, `()`, `[]`, `{}`, `set()`
* Objekte von benutzerdefinierten Klassen, die bei `len(obj)` 0 zurückgeben

> **object.__bool__(self)** <br>
>  Called to implement truth value testing and the built-in operation bool(); should return False or True. When this method is not defined, __len__() is called, if it is defined, and the object is considered true if its result is nonzero. If a class defines neither __len__() nor __bool__(), all its instances are considered true.

[Dokumentation](https://docs.python.org/3/reference/datamodel.html#object.__bool__)

In [ ]:
def find_truthyness(var):
    try:
        return var.__bool__()
    except AttributeError:
        try:
            return var.__len__() != 0
        except AttributeError:
            return True

In [ ]:
print(find_truthyness(""))
print(find_truthyness("asdf"), '\n')

print(find_truthyness([]))
print(find_truthyness([1, 2]), '\n')

print(find_truthyness(0))  #int.__bool__(0)
print(find_truthyness(1))  #int.__bool__(1)

Weitere Informationen hier: https://docs.python.org/3/reference/datamodel.html#special-method-names

## Properties

Andere Sprachen definieren oft *getter* und *setter*, um den Zugriff auf Objektattribute zu beschränken. In Python können wir Getter- und Setter-Logik mit `Properties` hinzufügen.

In [ ]:
class Triple:
    def __init__(self, num1, num2, num3):
        self._nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return f"Triple({self.nums[0]}, {self.nums[1]}, {self.nums[2]})"
    
a = Triple(1, 2, 3)

In [ ]:
class Triple():
    def __init__(self, num1, num2, num3):
        self._nums = num1, num2, num3
      
    @property
    def nums(self):
        return self._nums

    
a = Triple(1, 2, 3)
a.nums

Standardmäßig können wir Attribute, die über Eigenschaften deklariert sind, nicht zuweisen.

In [ ]:
a.nums = 10, 11, 12

Aber wir können einen Setter mit einem anderen Dekorator hinzufügen. Dies ist nützlich für die Einbeziehung von Validierungslogik.

In [ ]:
class Triple():
    def __init__(self, num1, num2, num3):
        self._nums = num1, num2, num3
      
    @property
    def nums(self):
        return self._nums
    
    @nums.setter
    def nums(self, value):
        if len(value) == 3:
            self._nums = value
        else:
            raise ValueError("Three values are required to set the data.")
    
a = Triple(1, 2, 3)
a.nums = (4, 5, 6)
a.nums

Mit `properties` können wir Getter- und Setter-Logik hinzufügen, ohne dass diese in den Schnittstellen unserer Objekte auftauchen. Das bedeutet auch, dass Sie Ihre Klassen zunächst mit einfachen Attributen schreiben und später bei Bedarf Getter und Setter hinzufügen können.

## Factory-Methoden

Manchmal möchten wir unsere Objekte auf unterschiedliche Weise initialisieren können. Ein klassisches Muster der objektorientierten Programmierung ist die `Factory`. Eine `Factory` hat nur den Zweck, andere Objekte zu initialisieren. In Python brauchen wir dieses Muster nicht wirklich, sondern wir können stattdessen *Factory-Methoden* verwenden, um unser Objekt auf unterschiedliche Weise zu initialisieren. Dazu können wir Klassenmethoden, die die Klasse statt der Instanz als erstes Argument nehmen.

In [ ]:
class Triple():
    def __init__(self, num1, num2, num3):
        self._nums = num1, num2, num3
    
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return f"Triple({self.nums[0]}, {self.nums[1]}, {self.nums[2]})"
    
    @property
    def nums(self):
        return self._nums
    
    @classmethod
    def from_value(cls, num):
        return cls(num, num, num)

Triple.from_value(3)

## Decorators

Decorators sind Funktionen, die die Funktionalität anderer Funktionen oder Klassen verändern. Dies sollte in der Regel auf transparente Weise geschehen, d.h. die Schnittstelle der ursprünglichen Funktion bleibt gleich, während die Funktionalität um sie herum hinzugefügt wird.

In [ ]:
def substract(x, y):
    return x - y

def decorated_substract(*args, **kwargs):
    print('~~~ result of', substract.__name__, '~~~')
    result = substract(*args, **kwargs)              
    print(result)                               
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~')        
    return result    
    
decorated_substract(5, 2);

Dies erzeugt jedoch nur eine neue Funktion, die ein geändertes Verhalten der Substract-Funktion enthält. Was ist, wenn wir das Verhalten beliebiger Funktionen ändern wollen?

In [ ]:
def substract(x, y):
    return x - y

def add(x, y):
    return x + y

def decorated(func, *args, **kwargs):
    result = func(*args, **kwargs)              
    print('~~~ result of', func.__name__, '~~~')
    print(result)                               
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~')        
    return result    
    
decorated(add, 5, 2)

In [ ]:
decorated(substract, 5, 2)

Damit sind wir noch nicht einverstanden, denn wir wollen das Verhalten von `add` selbst ändern!

In [ ]:
def print_decorator(func):                           # func is the method which will be decorated by this
        
    print("This occurs when we re-define the function")
    
    #if we define function = decorated(function), the new function will be this:
    
    def inner(*args, **kwargs):                      # we define a new function here, taking any parameters...
        result = func(*args, **kwargs)               # which, when called, executes the original function with these parameters...
        print('~~~ result of', func.__name__, '~~~') # prints name of original funciton...
        print(result)                                # prints the result of the function...
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~')         # some lines...
        return result                                # and returns that result of that function 
    
    return inner   # the new function is this inner function!

In [ ]:
decorated_add = print_decorator(add)

In [ ]:
decorated_add(3, 5)

In [ ]:
add = print_decorator(add)
add(3,5)

Python bietet eine Syntax für die Zuweisung `function = decorated(function)`. Dies ist jedoch nur *syntactic sugar* für den direkten Aufruf des Dekorators. 

In [ ]:
@print_decorator #multiply = print_decorator(multiply)  
def multiply(x, y):
    return x * y

multiply(3, 5)
multiply(4, 5)

Ein weiteres Beispiel:

In [ ]:
def bold(fn):
    """wraps the result of a function such that it's bold"""
    def wrapped():
        return f"<b>{fn()}</b>"
    return wrapped


@bold #hello = bold(hello)
def hello():
    """prints 'hello world'"""
    return "hello world"

hello()

In [ ]:
from IPython.display import HTML
HTML(hello())

Wir können sogar Decorators verketten!

In [ ]:
def bold(fn):
    """wraps the result of a function such that it's bold"""
    def wrapped():
        return f"<b>{fn()}</b>"
    return wrapped

def italic(fn):
    """wraps the result of a function such that it's italics"""
    def wrapped():
        return f"<i>{fn()}</i>"
    return wrapped

@bold #hello = bold(hello)
@italic #hello = italic(bold(hello))
def hello():
    """prints 'hello world'"""
    return "hello world"

hello()

In [ ]:
hello?

Das war's dann auch schon fast mit dem Grundwissen über Dekoratoren! Es gibt nur noch eine wichtige Sache: Wenn wir die ursprüngliche Funktion durch die dekorierte Version ersetzen, verlieren wir alle Informationen der ursprünglichen Funktion, wie ihren Docstring, Informationen über Argumente, usw. Um das zu kompensieren, verwenden wir *einen weiteren Dekorator*, nämlich `functools.wraps`. Dieser kopiert einfach den Docstring der Originalfunktion in die neue Funktion.

In [ ]:
from functools import wraps
from IPython.display import HTML

def html(fn):
    @wraps(fn)
    def wrapped():
        return HTML(fn())
    return wrapped


def bold(fn):
    @wraps(fn)
    def wrapped():
        return f"<b>{fn()}</b>"
    return wrapped

def italic(fn):
    @wraps(fn)
    def wrapped():
        return f"<i>{fn()}</i>"
    return wrapped

@html
@bold
@italic
def hello():
    """prints 'hello world'"""
    return "hello world"

hello()

In [ ]:
hello?

**Wiederholen Sie für sich das Konzept der Decorator!**

Bearbeiten Sie inbesondere die folgende **Übung** und schreiben Sie die Antwort am Ende der Bearbeitungszeit in den Chat: 

Definieren und wenden Sie einen Dekorator an, der eine Zeichenkette rot erscheinen lässt. Sie können dies erreichen, indem Sie die Zeichenkette in `<span style='color: red'> str </span>` wrappen.

## Module
### Funktionen aus Modulen importieren

Die beruhigende Nachricht ist: viele Probleme wurden schon gelöst. Für häufige Aufgaben, wie bspw. das Sortieren von Listen, existieren sogar hochoptimierte und getestete Lösungen, die wir tunlichst verwenden sollten, anstatt unsere eigene zu schreiben!

Abgesehen von einigen grundlegenden Datentypen und Funktionen wie `print` oder `len` sind diese Funktionen nicht in der Python Standard Library enthalten sondern in **Modulen** ausgelagert. Mit der folgenden Syntax können wir ein Modul **importieren**, um auf die enthaltenen Funktionen zugreifen zu können:

```python
import module
module.function_name()
```

Häufig verwendeten Modulen können wir einen abgekürzten Namen geben:

```python
import module as m
m.function_name()
```

Wir können auch nur einzelne Funktionen eines Moduls importieren:

```python
from module import function_name
function_name()
```

Anstatt die Funktion `factorial` aus der obigen Aufgabe selbst zu schreiben, können wir nun einfach die gleichnamige Funktion aus dem Modul `math` verwenden:

> Es gibt natürlich nicht nur Module für die wissenschaftliche Anwendung. Python wird höchst vielseitig eingesetzt, sodass du bspw. auch
> - einen [Webservice programmieren](http://www.djangoproject.com) oder
> - ein [Spiel entwickeln](http://www.pygame.org) kannst!

Nun kannst du vollständige Programme schreiben und Funktionen aus Modulen verwenden. Erinnere dich daran - du musst nicht alles selbst schreiben! Baue lieber auf der Vorarbeit von schlauen Entwicklern auf der ganzen Welt auf, die schon hochoptimierte und getestete Lösungen für viele Probleme geschrieben haben. [giyf](http://www.google.de).

In den nächsten drei Lektionen lernen wir die Grundlagen jeweils eines Moduls, das in der wissenschaftlichen Programmierung mit Python allgegenwärtig ist und beginnen mit dem Numerik-Modul _Numpy_.

**Recherchieren Sie jetzt selber nach interessanten [Modulen](https://docs.python.org/3/py-modindex.html) und Bibliotheken, die Ihnen die Programmierung in Python erleichtern könnten! Es gibt sehr viel und so gut wie alles ist in Pythin einfach installierbar, importierbar und nutzbar. Anschließend können Sie direkt in das Übungsblatt einsteigen.**

Hier ist das Übungsblatt zu diesem Notebook: [**03 - Übungsaufgaben Objektorientierung und Sonstiges**](03_uebungsaufgaben_objektorientierung_sonstiges.ipynb)

---

Wahlpflichtfach Künstliche Intelligenz I: Praktikum